```{contents}
```
## Tool Sandboxing

**Tool sandboxing** in LangGraph is a production-grade safety mechanism that **isolates, restricts, monitors, and governs tool execution** inside an LLM workflow.
It prevents models and agents from performing **unsafe, unauthorized, or destructive actions** while still enabling powerful automation.

---

### **1. Motivation**

LLM agents can:

* Execute shell commands
* Call APIs
* Modify files
* Access databases
* Perform transactions

Without sandboxing, a single hallucinated tool call can cause **data loss, security breaches, or infrastructure damage**.

Tool sandboxing enforces **controlled autonomy**.

---

### **2. Conceptual Architecture**

```
LLM / Agent
   ↓
Tool Policy Layer  ←── Permissions / Rules
   ↓
Sandbox Executor   ←── Resource limits, isolation
   ↓
Actual Tool / API
```

Every tool invocation passes through **policy + sandbox** before execution.

---

### **3. Core Components**

| Component           | Role                           |
| ------------------- | ------------------------------ |
| Tool Registry       | Whitelist of allowed tools     |
| Tool Policy Engine  | Authorization & validation     |
| Sandbox Executor    | Isolated execution environment |
| Resource Controller | CPU, memory, time limits       |
| Audit Logger        | Full traceability              |
| Approval Gate       | Optional human verification    |

---

### **4. Tool Registration with Policies**

```python
from langchain.tools import tool

@tool
def delete_file(path: str) -> str:
    """Delete a file"""
    ...
```

Attach policy:

```python
tool_policy = {
    "delete_file": {
        "requires_human_approval": True,
        "allowed_paths": ["/safe_zone/"],
        "max_calls": 1
    }
}
```

---

### **5. Sandboxed Execution Node**

```python
def sandbox_node(state):
    tool_name = state["tool"]
    args = state["args"]

    if not policy_engine.is_allowed(tool_name, args):
        raise SecurityError("Tool blocked")

    result = sandbox_executor.run(tool_name, args)
    return {"tool_result": result}
```

---

### **6. Common Sandbox Controls**

| Control              | Purpose                                    |
| -------------------- | ------------------------------------------ |
| Filesystem isolation | Prevent reading/writing outside safe paths |
| Network restrictions | Block internet / internal services         |
| Process isolation    | No access to host OS                       |
| Time limits          | Prevent infinite execution                 |
| Memory limits        | Prevent crashes                            |
| Call quotas          | Prevent abuse                              |

---

### **7. Integration with LangGraph Flow**

```python
builder.add_node("tool_sandbox", sandbox_node)
builder.add_edge("agent", "tool_sandbox")
builder.add_edge("tool_sandbox", "agent")
```

The agent **cannot bypass** the sandbox.

---

### **8. Human-in-the-Loop Safeguards**

```python
if policy.requires_approval(tool_name):
    return {"interrupt": True, "request": "Approve delete?"}
```

Human approves → execution resumes.

---

### **9. Production Variants**

| Variant             | Use Case                |
| ------------------- | ----------------------- |
| Local sandbox       | Safe testing            |
| Container sandbox   | Cloud deployments       |
| VM sandbox          | High-risk tools         |
| Serverless sandbox  | Scalable automation     |
| Policy-only sandbox | Low-risk internal tools |

---

### **10. Failure Handling**

| Scenario            | Response     |
| ------------------- | ------------ |
| Policy violation    | Block + log  |
| Timeout             | Abort        |
| Exception           | Rollback     |
| Suspicious behavior | Freeze graph |
| Human rejection     | Reroute      |

---

### **11. Why Tool Sandboxing Is Essential for Agents**

Without sandboxing:

> **Agents are dangerous programs.**

With sandboxing:

> **Agents become controlled autonomous systems.**

---

### **12. Mental Model**

Tool sandboxing transforms LLM agents from:

> *Unrestricted script execution*
> into
> **Secure, auditable, governed automation systems**



### Demonstration

In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict
import time

# -------------------------------
# 1. State Definition
# -------------------------------

class State(TypedDict):
    tool: str
    args: dict
    approved: bool
    result: str

# -------------------------------
# 2. Tool & Sandbox
# -------------------------------

def delete_file(path: str):
    return f"Simulated delete of {path}"

# Tool registry
TOOLS = {"delete_file": delete_file}

# Tool policy
POLICY = {
    "delete_file": {
        "requires_human_approval": True,
        "allowed_paths": ["/safe_zone/"],
        "timeout": 2
    }
}

# -------------------------------
# 3. Policy Engine
# -------------------------------

def policy_check(tool, args):
    rule = POLICY.get(tool)
    if not rule:
        return False, "Tool not registered"

    if rule["requires_human_approval"] and not args.get("approved"):
        return False, "Approval required"

    if not any(args["path"].startswith(p) for p in rule["allowed_paths"]):
        return False, "Path violation"

    return True, ""

# -------------------------------
# 4. Sandbox Executor
# -------------------------------

def sandbox_node(state: State):
    tool = state["tool"]
    args = state["args"]

    allowed, reason = policy_check(tool, {**args, "approved": state["approved"]})

    if not allowed:
        return {"result": f"BLOCKED: {reason}"}

    start = time.time()
    output = TOOLS[tool](**args)

    if time.time() - start > POLICY[tool]["timeout"]:
        return {"result": "TIMEOUT"}

    return {"result": output}

# -------------------------------
# 5. Human Approval Node
# -------------------------------

def approval_node(state: State):
    # Simulated human decision
    return {"approved": True}

# -------------------------------
# 6. Graph Construction
# -------------------------------

builder = StateGraph(State)

builder.add_node("approve", approval_node)
builder.add_node("sandbox", sandbox_node)

builder.set_entry_point("approve")
builder.add_edge("approve", "sandbox")
builder.add_edge("sandbox", END)

graph = builder.compile()

# -------------------------------
# 7. Execution
# -------------------------------

result = graph.invoke({
    "tool": "delete_file",
    "args": {"path": "/safe_zone/file.txt"},
    "approved": False,
    "result": ""
})

print(result)


{'tool': 'delete_file', 'args': {'path': '/safe_zone/file.txt'}, 'approved': True, 'result': 'Simulated delete of /safe_zone/file.txt'}
